In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data Cleaning and Processing

## 0.1 Filling missing values

In [48]:
data = pd.read_csv('../data/raw_data.csv')
data.columns

Index(['phone_brand', 'phone_model', 'store', 'price', 'currency', 'price_USD',
       'storage', 'ram', 'Launch', 'Dimensions', 'Weight', 'Display_Type',
       'Display_Size', 'Display_Resolution', 'OS', 'NFC', 'USB', 'BATTERY',
       'Features_Sensors', 'Colors', 'Video', 'Chipset', 'CPU', 'GPU', 'Year',
       'Foldable', 'PPI_Density', 'quantile_10', 'quantile_50', 'quantile_90',
       'price_range'],
      dtype='object')

In [49]:
data.isnull().sum()[data.isnull().sum() > 0]
data[data['Weight'].isnull()]

# Drop Tab model, which not belongs to phone
tab_index = data['phone_model'].apply(lambda x: 'Tab' in x)
data = data[~tab_index]

In [50]:
# Filling the missing values with information from the internet
data.loc[1042, 'Weight'] = 199.0
data.loc[1151, 'Weight'] = 202.0
data.loc[1395, 'Weight'] = 186.0
data.isnull().sum()[data.isnull().sum() > 0]

Series([], dtype: int64)

In [51]:
data['Launch'] = pd.to_datetime(data['Launch'])


## 0.2 Processing some columns

### 0.2.1 Averge the price of the same product (from different sellers)

In [52]:
info_cols = ['phone_brand', 'phone_model', 'storage', 'ram', 'Launch', 'Dimensions', 'Weight', 'Display_Type', 'Display_Resolution', 'NFC', 'USB', 'BATTERY', 'Features_Sensors', 'Colors', 'Video', 'Chipset', 'CPU', 'GPU', 'Year', 'Foldable', 'PPI_Density']
data_reduced = data[info_cols + ['price_USD']]
def compress_price_info(data):
    average_price = data.groupby(info_cols).mean().reset_index()
    return average_price
data_reduced = compress_price_info(data_reduced)
len(data_reduced)

706

### 0.2.2 Handle `dimension` column
1. split `dimension` into `length`, `width`, `height`
2. calculate the `display_ratio` = `length` / `width`

In [53]:
def transform_dimension(dim_str):
    def tofloat(value):
        try:
            return float(value)
        except ValueError:
            return False
    words = dim_str.replace('/', ' ').split()
    dim = []
    if len(words) < 3:
        print(words)
        TypeError('Dimension string is not in the correct format')
    for i in range(len(words)):
        digit = tofloat(words[i])
        if digit:
            dim.append(digit)
            if len(dim) == 3:
                break
    # ratio rounded to 2 decimal places
    ratio = round(dim[0] / dim[1], 1)
    dim.append(ratio)
    assert len(dim) == 4, dim
    return dim

## Create 4 new columns for the dimensions
def create_new_dimensions(data):
    data['Length'], data['Width'], data['Height'], data['Ratio'] = zip(*data['Dimensions'].map(transform_dimension))
    return data
data_reduced = create_new_dimensions(data_reduced)

In [54]:
if all(data_reduced['Launch'].dt.year == data_reduced['Year']): # This is True
    data_reduced = data_reduced.drop(columns=['Year'])
    

### 0.2.3 Extract Display technology, refresh rate from `display` column

In [55]:
def transform_display_info(info):
    screen_tech = None
    if 'AMOLED' in info:
        screen_tech = 'AMOLED'
    elif 'OLED' in info:
        screen_tech = 'OLED'
    elif 'LCD' in info:
        screen_tech = 'LCD'
    
    screen_refresh_rate = None
    info = info.lower()
    if 'hz' in info:
        idx = info.index('hz')
        screen_refresh_rate = int(info[idx-3:idx])
    return screen_tech, screen_refresh_rate

def create_new_display_info(data):
    data['Screen_Tech'], data['Screen_Refresh_Rate'] = zip(*data['Display_Type'].map(transform_display_info))
    return data

data_reduced = create_new_display_info(data_reduced)
print(data_reduced['Screen_Refresh_Rate'].isnull().sum())
data_reduced.fillna({'Screen_Refresh_Rate': 60}, inplace=True)

176


### 0.2.4 Extract USB version from `usb` column


In [56]:
def transform_usb_info(info):
    usb_type = None
    if 'Type-C' in info:
        usb_type = 'Type-C'
    elif 'micro' in info:
        usb_type = 'Micro-USB'
    elif 'Lightning' in info:
        usb_type = 'Lightning'
    
    usb_version = None
    if '3' in info:
        usb_version = 3
    elif '2' in info:
        usb_version = 2
    return usb_type, usb_version

def create_new_usb_info(data):
    data['USB_Type'], data['USB_Version'] = zip(*data['USB'].map(transform_usb_info))
    return data

data_reduced = create_new_usb_info(data_reduced)
print(data_reduced['USB_Type'].isnull().sum())
print(data_reduced['USB_Version'].isnull().sum())
data_reduced.fillna({'USB_Type': 'Type-C', 'USB_Version': 2}, inplace=True)

0
21


### 0.2.5 Extract color choices from `color` column

In [57]:
data_reduced['Num_Color_Options'] = data_reduced['Colors'].map(lambda x: len(x.split(',')))

### 0.2.6 Extract the maximum video recording resolution from `video` column

In [58]:
def transform_video_info(info):
    info = info.split(',')
    supported_resolutions = []
    for i in range(len(info)):
        if '8K' in info[i]:
            if '30' in info[i] or '24' in info[i]:
                supported_resolutions.append('8K@24/30fps')
            else:
                ValueError('8K resolution format is not recognized')
        if '4K' in info[i]:
            if '30' in info[i] or '24' in info[i]:
                supported_resolutions.append('4K@24/30fps')
            elif '60' in info[i]:
                supported_resolutions.append('4K@60fps')
            elif '120' in info[i]:
                supported_resolutions.append('4K@120fps')
            else:
                supported_resolutions.append('4K@24/30fps')
        if '1080' in info[i]:
            if '30' in info[i] or '24' in info[i]:
                supported_resolutions.append('1080p@24/30fps')
            elif '60' in info[i]:
                supported_resolutions.append('1080p@60fps')
            elif '120' in info[i]:
                supported_resolutions.append('1080p@120fps')
            elif '240' in info[i]:
                supported_resolutions.append('1080p@240fps')
            else:
                supported_resolutions.append('1080p@24/30fps')
        if '720' in info[i] and supported_resolutions is None:
            if '30' in info[i] or '24' in info[i]:
                supported_resolutions.append('720p@24/30fps')
            elif '60' in info[i]:
                supported_resolutions.append('720p@60fps')
            elif '120' in info[i]:
                supported_resolutions.append('720p@120fps')
            else:
                ValueError('720p resolution format is not recognized')
        if not supported_resolutions:
            supported_resolutions.append('1080p@24/30fps')
    return supported_resolutions

def create_new_video_info(data):
    data['Supported_Video_Resolutions'] = data['Video'].map(transform_video_info)
    return data

data_reduced = create_new_video_info(data_reduced)
print(data_reduced['Supported_Video_Resolutions'].isnull().sum())

0


### 0.2.7 Extract chipset information from `chipset` column
Manufacurer + nm

In [59]:
def transform_chipset_info(info):
    manufacturer = info.split()[0]
    if manufacturer == 'Mediatek':
        manufacturer = 'MediaTek'
    nm = None
    # format: Manufacturer _submodel_ (Nnm)
    if 'nm' in info:
        nm = int(info[info.index('(')+1:info.index('nm')])
    return manufacturer, nm

def create_new_chipset_info(data):
    data['Chipset_Manufacturer'], data['Chipset_(nm)'] = zip(*data['Chipset'].map(transform_chipset_info))
    return data

data_reduced.loc[105,'Chipset'] = 'Qualcomm Snapdragon 662 (11nm)'
data_reduced.loc[[588, 688, 689] ,'Chipset'] = 'Mediatek Helio G99 Ultra (6nm)'
data_reduced.loc[[622, 623],'Chipset'] = 'Mediatek Helio G81 Ultra (12nm)'
data_reduced.loc[131,'Chipset'] = 'Kirin 9000S1 (7nm)'
data_reduced = create_new_chipset_info(data_reduced)
print(data_reduced['Chipset_Manufacturer'].isnull().sum())
print(data_reduced['Chipset_(nm)'].isnull().sum())

0
5


### 0.2.8 Extract the number of cores from `cpu` column

In [60]:
def transform_cpu_info(info):
    if 'Hexa' in info or '6-core' in info:
        return 6
    elif 'Octa' in info or '8-core' in info or info == '8':
        return 8
    elif 'Nona' in info or '9-core' in info:
        return 9
    elif 'Deca' in info or '10-core' in info:
        return 10
    elif 'Quad' in info or '4-core' in info:
        return 4
    else:
        print(info)
        ValueError('Number of cores is not recognized')

def create_new_cpu_info(data):
    data['Num_CPU_Cores'] = data['CPU'].map(transform_cpu_info)
    return data

data_reduced = create_new_cpu_info(data_reduced)
print(data_reduced['Num_CPU_Cores'].isnull().sum())


0


## Save the preprocessed data

In [61]:
data_reduced.columns

Index(['phone_brand', 'phone_model', 'storage', 'ram', 'Launch', 'Dimensions',
       'Weight', 'Display_Type', 'Display_Resolution', 'NFC', 'USB', 'BATTERY',
       'Features_Sensors', 'Colors', 'Video', 'Chipset', 'CPU', 'GPU',
       'Foldable', 'PPI_Density', 'price_USD', 'Length', 'Width', 'Height',
       'Ratio', 'Screen_Tech', 'Screen_Refresh_Rate', 'USB_Type',
       'USB_Version', 'Num_Color_Options', 'Supported_Video_Resolutions',
       'Chipset_Manufacturer', 'Chipset_(nm)', 'Num_CPU_Cores'],
      dtype='object')

In [62]:
# Drop the columns that are already extracted
columns_to_keep = ['phone_brand', 'phone_model', 'storage', 'ram', 'Launch',
       'Weight', 'Display_Resolution', 'NFC', 'BATTERY',
       'Features_Sensors', 'GPU',
       'Foldable', 'PPI_Density', 'price_USD', 'Length', 'Width', 'Height',
       'Ratio', 'Screen_Tech', 'Screen_Refresh_Rate', 'USB_Type',
       'USB_Version', 'Num_Color_Options', 'Supported_Video_Resolutions',
       'Chipset_Manufacturer', 'Chipset_(nm)', 'Num_CPU_Cores']
data_reduced = data_reduced[columns_to_keep]

In [63]:
data_reduced.to_csv('../data/cleaned_data.csv', index=False)

In [64]:
import pandas as pd
data_reduced = pd.read_csv('../data/cleaned_data.csv')